# Data Modeling

In [2]:
import pandas as pd

In [3]:
# Reading the
listing_dataframe = pd.read_csv("cleaned_listing_data.csv")

In [6]:
listing_dataframe.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
number_of_reviews_ltm               int64
reviewed                            int64
dtype: object

The columns such as host_name, name , neighhbourhood doesn't provide much predictive power, and also has high cardinality. So dropping them for data modelling. Similarly, host_id and id don't have much predictive

In [13]:
X = listing_dataframe.drop(['id', 'host_id', 'name', 'host_name', 'neighbourhood', 'price'],axis=1)
y = listing_dataframe['price']

In [14]:
X.head()

,neighbourhood_group,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,reviewed
0,Manhattan,40.75356,-73.98559,Entire home/apt,30,49,0.30,3,314,1,1
1,Brooklyn,40.68535,-73.95512,Private room,30,50,0.30,2,365,0,1
2,Manhattan,40.80380,-73.96751,Private room,2,118,0.72,1,0,0,1
3,Manhattan,40.76457,-73.98317,Private room,2,575,3.41,1,106,52,1
4,Brooklyn,40.66265,-73.99454,Entire home/apt,60,3,0.03,1,181,1,1


In [17]:
# Converting neighbourhood_group and room_type to one-hot encoding
encoded_neighbourhood_group = pd.get_dummies(X["neighbourhood_group"])
encoded_room_type = pd.get_dummies(X["room_type"])

In [20]:
X = pd.concat([X, encoded_neighbourhood_group, encoded_room_type], axis=1 )

In [24]:
X.drop(["neighbourhood_group","room_type"], axis=1, inplace=True)

In [25]:
X.columns

Index(['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'reviewed', 'Bronx',
       'Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'Entire home/apt',
       'Hotel room', 'Private room', 'Shared room'],
      dtype='object')